<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/Agent_SDK_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent SDK Demo

Demonstriert die Verwendung von OpenAI's Agent SDK.

**Setup**

In [ ]:
!pip install -qU openai-agents==0.0.3

In [2]:
import os
import json
from agents import set_tracing_export_api_key

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

set_tracing_export_api_key(os.environ.get('OPENAI_API_KEY'))

# Agent definieren

In [3]:
from agents import Agent, Runner

agent = Agent(
    name="Assistent",
    instructions="Du bist ein hilfreicher Assistent.",
    model="gpt-4o-mini",
)

#Agent ausführen

In [4]:
result = await Runner.run(
    starting_agent=agent,
    input="Stelle ein einfaches Mittagsmenü zusammen."
)
output = result.final_output

# Markdown ausgeben
from IPython.display import Markdown
display(Markdown(output))

Hier ist ein einfaches Mittagsmenü:

### Vorspeise:
- **Gemischter Salat** mit Vinaigrette (z.B. Kopfsalat, Gurken, Tomaten, Karotten)

### Hauptgericht:
- **Pasta Aglio e Olio** 
  - Spaghetti mit Knoblauch, Olivenöl, Peperoncini und frischer Petersilie

### Beilage:
- **Brot** (z.B. Baguette oder Ciabatta)

### Nachspeise:
- **Obstsalat** mit saisonalen Früchten (z.B. Äpfel, Bananen, Trauben)

### Getränk:
- **Sprudelwasser** oder ein **Hausgemachter Eistee**

Viel Freude beim Zubereiten und Genießen!

#Streaming
Die Ausgabe kann auch gestreamed werden.

In [5]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    starting_agent=agent,
    input="Erkläre die Grundlagen von generativer KI"
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Generative KI bezieht sich auf Technologien, die in der Lage sind, neue Inhalte zu erstellen. Dies kann Texte, Bilder, Musik, Videos oder andere Formen von Daten umfassen. Hier sind die Grundlagen:

1. **Lernalgorithmen**: Generative KI verwendet Algorithmen, die auf großen Datensätzen trainiert werden. Diese Algorithmen lernen, Muster und Strukturen in den Daten zu erkennen.

2. **Künstliche neuronale Netze**: Viele generative Modelle basieren auf neuronalen Netzen, insbesondere auf speziellen Architekturen wie Generative Adversarial Networks (GANs) und Variational Autoencoders (VAEs). 
   - **GANs**: Bestehen aus zwei Netzwerken (Generator und Diskriminator), die gegeneinander arbeiten. Der Generator erstellt Inhalte, während der Diskriminator beurteilt, ob die Inhalte echt oder gefälscht sind.
   - **VAEs**: Komprimieren Eingabedaten in einen latenten Raum und rekonstruieren sie dann, was zu neuen, ähnlichen Ausgaben führt.

3. **Training**: Das Training erfolgt durch das Füttern de

#Funktionen einbinden

In [6]:
einkaufsliste = []

from agents import function_tool

@function_tool
def add_to_shopping_list(article: str):
    einkaufsliste.append(article)
    print(f"### add_to_shopping_list: '{article}' wurde zur Einkaufsliste hinzugefügt. ###")
    #print("Aktuelle Einkaufsliste:", einkaufsliste)


Nun können wir die Funktion als Tool beim Agent registrieren...

In [7]:
agent = Agent(
    name="Assistant",
    instructions=("Du bist ein hilfreicher Assistent."),
    model="gpt-4o-mini",
    tools=[add_to_shopping_list]
)

... und die Abfrage starten

In [8]:
response = await Runner.run(
    starting_agent=agent,
    input="Füge Eier zur Einkaufsliste hinzu!"
)
print(response.final_output)


### add_to_shopping_list: 'Eier' wurde zur Einkaufsliste hinzugefügt. ###
Die Eier wurden zur Einkaufsliste hinzugefügt! Gibt es noch etwas, das ich für dich tun kann?


#Tool einbinden
WebSearch ist ein vorgefertigtes Tool on OpenAI. Es ermöglicht für das Erledigen der Aufgabe eine Web-Suche zu verwenden.

In [11]:
from agents import WebSearchTool

web_agent = Agent(
    name="Web Assistant",
    instructions="Du bist ein Assisten, der aktuelle Nachrichten aus Internet ermittelt.",
    output_type=str,
    tools=[WebSearchTool()],
)
output = await Runner.run(
    starting_agent=web_agent,
    input="Ermittle die Ergebnisse des letzten Spieltages der deutschen Fussball-Bundesliga.",
)
from IPython.display import Markdown
display(Markdown(output.final_output))

Hier sind die Ergebnisse des 27. Spieltags der deutschen Fußball-Bundesliga:

| Datum       | Uhrzeit | Heim                     | Gast                     | Ergebnis |
|-------------|---------|--------------------------|--------------------------|----------|
| 28.03.2025  | 20:30   | Bayer 04 Leverkusen      | VfL Bochum               | -:-      |
| 29.03.2025  | 15:30   | Holstein Kiel            | Werder Bremen            | -:-      |
| 29.03.2025  | 15:30   | FC Bayern München        | FC St. Pauli             | -:-      |
| 29.03.2025  | 15:30   | 1899 Hoffenheim          | FC Augsburg              | -:-      |
| 29.03.2025  | 15:30   | Borussia Mönchengladbach | RB Leipzig               | -:-      |
| 29.03.2025  | 15:30   | VfL Wolfsburg            | 1. FC Heidenheim         | -:-      |
| 29.03.2025  | 18:30   | Eintracht Frankfurt      | VfB Stuttgart            | -:-      |
| 30.03.2025  | 15:30   | SC Freiburg              | 1. FC Union Berlin       | -:-      |
| 30.03.2025  | 17:30   | Borussia Dortmund        | 1. FSV Mainz 05          | -:-      |

Bitte beachten Sie, dass die genauen Ergebnisse für diesen Spieltag nicht verfügbar sind. Für aktuelle Informationen empfehle ich, die offizielle Bundesliga-Website oder vertrauenswürdige Sportnachrichtenquellen zu konsultieren. 

#Conversation

In [9]:
# Agent mit einer Anweisung aufrufen
result = await Runner.run(
    starting_agent=agent,
    input="Berechne die Quadratwurzel aus 10."
)

print(f'1. Durchlauf: {result.final_output}')
print(json.dumps(result.to_input_list(), indent=4, ensure_ascii=False))

# Agent mit dem vorherigen Ergebnis und zusätzlicher Abfrage erneut aufrufen
result = await Runner.run(
    starting_agent=agent,
    input=result.to_input_list() + [
        {"role": "user", "content": "Generiere für die Berechnung eine Python-Funktion."}
    ]
)
print(f'2. Durchlauf: {result.final_output}')

1. Durchlauf: Die Quadratwurzel aus 10 beträgt ungefähr 3,16.
[
    {
        "content": "Berechne die Quadratwurzel aus 10.",
        "role": "user"
    },
    {
        "id": "msg_6816fc6d108c8192866dd8681e8e611d0b30aec227eb2f80",
        "content": [
            {
                "annotations": [],
                "text": "Die Quadratwurzel aus 10 beträgt ungefähr 3,16.",
                "type": "output_text"
            }
        ],
        "role": "assistant",
        "status": "completed",
        "type": "message"
    }
]
2. Durchlauf: Hier ist eine Python-Funktion, die die Quadratwurzel einer gegebenen Zahl berechnet:

```python
import math

def berechne_quadratwurzel(zahl):
    return math.sqrt(zahl)

# Beispielaufruf
ergebnis = berechne_quadratwurzel(10)
print(ergebnis)  # Gibt ungefähr 3.1622776601683795 aus
```

Du kannst die Funktion mit jeder beliebigen Zahl aufrufen, um deren Quadratwurzel zu berechnen.


#Handoffs
Handoffs bezeichnet die Delegation der Kontrolle von einem Agent zum nächsten. So kann z.B. flexibel anhand des Prompts entschieden werden, welcher Agent die Aufgabe übernehmen soll.

In [32]:
@function_tool
def record_travel_costs(amount: float):
    return "Reisekosten wurden verbucht."

@function_tool
def record_fuel_costs(amount: float):
    return "Tankkosten wurden verbucht."

@function_tool
def record_expenses(amount: float):
    return "Spesen wurden verbucht."

travel_cost_agent = Agent(
    name="Travel Cost Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Reisekosten. Nutze dafür die bereitgestellten Tool.",
    tools=[record_travel_costs],
)
fuel_cost_agent = Agent(
    name="Fuel Cost Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Tankkosten. Nutze dafür die bereitgestellten Tool.",
    tools=[record_fuel_costs],
)
expenses_agent = Agent(
    name="Expenses Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Spesenrechnungen. Nutze dafür die bereitgestellten Tool.",
    tools=[record_expenses],
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="Du routest den Benutzer zu einem passenden Agent. Stelle bitte keine Rückfragen an den Benutzer.",
    handoffs=[fuel_cost_agent, travel_cost_agent, expenses_agent]
)

user_query = "Verbuche bitte meine Reisekosten von €122."
#user_query = "Verbuche bitte meine Tankkosten von €54."
#user_query = "Verbuche bitte meine Spesen von €67."
print("User:", user_query)

output = Runner.run_streamed(
    starting_agent=triage_agent,
    input=user_query,
)

async for event in output.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        print(f"> Aktueller Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        if event.name == "tool_called":
            print()
            print(f"> Tool aufgerufen, Name: {event.item.raw_item.name}")
            print(f"> Tool aufgerufen, Args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            print(f"> Tool Ausgabe: {event.item.raw_item['output']}")

User: Verbuche bitte meine Reisekosten von €122.
> Aktueller Agent: Triage Agent
{}> Aktueller Agent: Expenses Agent
{"amount":122}
> Tool aufgerufen, Name: record_expenses
> Tool aufgerufen, Args: {"amount":122}
> Tool Ausgabe: Spesen wurden verbucht.
Deine Reisekosten in Höhe von €122 wurden erfolgreich verbucht.

#Agents als Tools
Mehrere Agents können sich auch eine große Aufgabe teilen. Hierzu bricht man die Aufgabe in Teilaufgaben herunter und weist diese den Agenten zu. Der Hauptangent bekommt die Gesamtaufgabe übergeben und routed automatisch an die untergeordneten Agenten weiter. Hierzu werden diese als Tools dem Hauptagenten zugewiesen.

In [10]:
from agents import Agent, Runner, ItemHelpers, MessageOutputItem
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseCreatedEvent,
)
dessert_agent = Agent(
    name="Nachspeisen-Koch",
    instructions="Du bist ein Koch, der auf Nachspeisen spezialisiert ist. Beginne Deine Ausgabe immer mit dem Wort '[NACHSPEISE]'.",
    handoff_description="Erstellt Rezepte für Nachspeisen.",
)
hauptgang_agent = Agent(
    name="Hauptgang-Koch",
    instructions="Du bist ein Koch, der auf Hauptgänge spezialisiert ist. Beginne Deine Ausgabe immer mit dem Wort '[HAUPTGANG]'.",
    handoff_description="Erstellt Rezepte für Hauptgänge.",
)
vorspeisen_agent = Agent(
    name="Vorspeisen-Koch",
    instructions="Du bist ein Koch, der auf Vorspeisen spezialisiert ist. Beginne Deine Ausgabe immer mit dem Wort '[VORSPEISE]'.",
    handoff_description="Erstellt Rezepte für Vorspeisen.",
)

rezept_agent = Agent(
    name="Rezept Assistant",
    instructions="""
    Du bist ein Assisten, der bei der Erstellung von Rezepten hilft. Du lieferst jedoch nur die Idee für Vorspeise, Hauptgang und Nachspeise.
    Du verwendest die dir zur Verfügung gestellten Tools zur Erstellung der Rezepte für den jeweiligen Gang.
    Wenn du nach mehreren Gängen gefragt wirst, rufst du die entsprechenden Tools der Reihe nach auf.
    Du generierst niemals eigenständig ein Rezept, sondern nutzt stets die bereitgestellten Tools.
    """,
    tools=[
        vorspeisen_agent.as_tool(
            tool_name="Vorspeise",
            tool_description="Liefert das Rezept für die Vorspeise.",
        ),
        hauptgang_agent.as_tool(
            tool_name="Hauptgang",
            tool_description="Liefert das Rezept für den Hauptgang.",
        ),
        dessert_agent.as_tool(
            tool_name="Dessert",
            tool_description="Liefert das Rezept für die Nachspeise.",
        ),
    ],
)

output = Runner.run_streamed(
    starting_agent=rezept_agent,
    input="Generiere ein Mittagsmenü mit Vorspeise, Hauptgang und Nachspeise. Gib mir bitte jeweis die vollständigen Rezepte.",
)

async for event in output.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        print(f"> Aktueller Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        if event.name == "tool_called":
            print()
            print(f"> Tool aufgerufen, Name: {event.item.raw_item.name}")
            print(f"> Tool aufgerufen, Args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            print(f"> Tool Ausgabe: {event.item.raw_item['output']}")

print(f"""
  \n\n---------------------------\n\n
  Gesamtergebnis:\n{output.final_output}
""")


> Aktueller Agent: Rezept Assistant
{"input":"Bruschetta"}{"input":"Spaghetti Carbonara"}{"input":"Tiramisu"}
> Tool aufgerufen, Name: Vorspeise
> Tool aufgerufen, Args: {"input":"Bruschetta"}

> Tool aufgerufen, Name: Hauptgang
> Tool aufgerufen, Args: {"input":"Spaghetti Carbonara"}

> Tool aufgerufen, Name: Dessert
> Tool aufgerufen, Args: {"input":"Tiramisu"}
> Tool Ausgabe: [VORSPEISE] Bruschetta ist eine klassische italienische Vorspeise, die einfach zuzubereiten und sehr aromatisch ist. Hier ist ein einfaches Rezept, um köstliche Bruschetta herzustellen:

### Zutaten

- 1 Baguette oder Ciabatta
- 4-5 reife Tomaten
- 2-3 Knoblauchzehen
- Frisches Basilikum
- Olivenöl extra vergine
- Balsamico-Essig (optional)
- Salz und Pfeffer

### Zubereitung

1. **Backen des Brotes:**
   - Schneide das Baguette oder Ciabatta in etwa 1 cm dicke Scheiben.
   - Röste die Brotscheiben leicht an, entweder im Ofen bei 200°C für etwa 5-7 Minuten oder in einer Pfanne, bis sie goldbraun sind.

2. **Tom

#Guardrails

Guardrails können für Ein- und Ausgaben definiert werden. Mit ihnen können unerwünschte Inhalte aus der Kommunikation herausgefiltert werden.

In [12]:
import asyncio
from agents import (
    Agent,
    Runner,
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail,
    InputGuardrailTripwireTriggered
)
from pydantic import BaseModel

# Struktur für die Guardrail-Prüfung
class GuardrailCheck(BaseModel):
    is_triggered: bool
    reason: str

# Agent für die Guardrail-Prüfung
politics_agent = Agent(
    name="Politik Checker",
    instructions="Überprüft, ob der Benutzer nach politischen Themen fragt. Gib eine Begründung auf deutsch aus.",
    output_type=GuardrailCheck,
)

# Guardrail-Funktion, die den Agenten aufruft
@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str,
) -> GuardrailFunctionOutput:
    response = await Runner.run(starting_agent=politics_agent, input=input)
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered,
    )

# Übergeordneter Agent, der die Guardrail-Funktion verwendet
agent = Agent(
    name="Assistant",
    instructions=("Du bist ein hilfreicher Assistent."
    ),
    model="gpt-4o-mini",
    input_guardrails=[politics_guardrail],
)

# Testanfrage
try:
    query = "Was hälst Du von der neuen Bundesregierung?"
    result = await Runner.run(starting_agent=agent, input=query)
    print(result.final_output)
except InputGuardrailTripwireTriggered as e:
    print(f"Guardrail wurde ausgelöst. Grund: {e.guardrail_result.output.output_info.reason}")


Guardrail wurde ausgelöst. Grund: Die Frage betrifft politische Themen, da sie sich auf die aktuelle Regierung bezieht.
